# Optimizing Model Parameters

https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root = 'data',                
    train = True,
    download = True,
    transform = ToTensor())

In [3]:
test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
    )

In [4]:
train_dataloader = DataLoader(training_data, batch_size = 128)
test_dataloader = DataLoader(test_data, batch_size = 128)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10))
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [6]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
lr = 2e-3
batch_size = 128
epochs = 5

In [8]:
loss_fn = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred,y)
        
        # backprop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (batch % 100 == 0):
            loss, current = loss.item(), (batch + 1)*len(X)
            print(f'loss: {loss:.7f}\t[{current/size}]')

In [11]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0,0
    
    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t}')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

Epoch 0
loss: 2.3115804	[0.0021333333333333334]
loss: 2.2762306	[0.21546666666666667]
loss: 2.2457299	[0.4288]
loss: 2.2215521	[0.6421333333333333]
loss: 2.1804790	[0.8554666666666667]
Test Error: 
 Accuracy: 42.7%, Avg loss: 2.153269 

Epoch 1
loss: 2.1593382	[0.0021333333333333334]
loss: 2.0896223	[0.21546666666666667]
loss: 2.0480008	[0.4288]
loss: 2.0019336	[0.6421333333333333]
loss: 1.9279108	[0.8554666666666667]
Test Error: 
 Accuracy: 54.3%, Avg loss: 1.872892 

Epoch 2
loss: 1.8711869	[0.0021333333333333334]
loss: 1.7497610	[0.21546666666666667]
loss: 1.6954808	[0.4288]
loss: 1.6254768	[0.6421333333333333]
loss: 1.5668992	[0.8554666666666667]
Test Error: 
 Accuracy: 62.4%, Avg loss: 1.508025 

Epoch 3
loss: 1.4927374	[0.0021333333333333334]
loss: 1.3861099	[0.21546666666666667]
loss: 1.3517952	[0.4288]
loss: 1.3056839	[0.6421333333333333]
loss: 1.2887959	[0.8554666666666667]
Test Error: 
 Accuracy: 64.2%, Avg loss: 1.248191 

Epoch 4
loss: 1.2253227	[0.0021333333333333334]
loss